In [1]:
import csv
import pandas as pd
import numpy as np
import urllib.parse
import urllib.request
from bs4 import BeautifulSoup

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def split_str_list(x, sep):
    """Function to split comma seperated string into array type data"""
    if type(x) == str and x != "" and x != "\\N":
        return x.split(sep)
    else:
        return None

In [4]:
m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url', 'unknown', 'Action',
          'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
          'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

movies = pd.read_csv('ml-100k/u.item', sep='|', names=m_cols, encoding='latin-1')

In [5]:
movies['release_date'] = movies['release_date'].apply(lambda x: str(x)[-4:])
max_year = np.nanmax(movies['release_date'].astype(float))
min_year = np.nanmin(movies['release_date'].astype(float))

In [6]:
scraped_data = {
    "movie_id": [],
    "movie_title": [],
    "movie_url": [],
}

In [7]:
for row in movies[['movie_id', 'title']].values:
    movie_id = row[0]
    movie_title = row[1]
    scraped_data['movie_id'].append(movie_id)
    scraped_data['movie_title'].append(movie_title)
    domain = 'http://www.imdb.com'
    search_url = domain + '/find?s=tt&q=' + urllib.parse.quote_plus(movie_title)
    with urllib.request.urlopen(search_url) as response:
        html = response.read()
        soup = BeautifulSoup(html, 'html.parser')
        # Get url of 1st search result
        try:
            title = soup.find('table', class_='findList').tr.a['href']
            movie_url = domain + title
            scraped_data['movie_url'].append(movie_url)
            # Ignore cases where search returns no results
        except AttributeError:
            scraped_data['movie_url'].append(np.nan)

In [8]:
scraped_df = pd.DataFrame(scraped_data)

We try again without the stuff between parenthesis for the missing links

In [10]:
movies['title_2'] = movies['title'].str.replace(r"\(.*\)","")

In [11]:
scraped_data_2 = {
    "movie_id": [],
    "movie_title": [],
    "movie_url_2": [],
}

In [12]:
for row in movies[['movie_id', 'title_2']].values:
    movie_id = row[0]
    movie_title = row[1]
    domain = 'http://www.imdb.com'
    search_url = domain + '/find?s=tt&q=' + urllib.parse.quote_plus(movie_title)
    scraped_data_2['movie_id'].append(row[0])
    scraped_data_2['movie_title'].append(row[1])
    with urllib.request.urlopen(search_url) as response:
        html = response.read()
        soup = BeautifulSoup(html, 'html.parser')
        # Get url of 1st search result
        try:
            title = soup.find('table', class_='findList').tr.a['href']
            movie_url = domain + title
            scraped_data_2['movie_url_2'].append(movie_url)
                # Ignore cases where search returns no results
        except AttributeError:
            scraped_data_2['movie_url_2'].append(np.nan)

In [13]:
scraped_df_2 = pd.DataFrame(scraped_data_2)

In [17]:
scraped_df['movie_url_2'] = scraped_df_2['movie_url_2']
scraped_df['movie_url'] = scraped_df['movie_url'].replace(np.nan, scraped_df['movie_url_2'])

In [24]:
scraped_df['id'] = scraped_df['movie_url'].apply(lambda x: split_str_list(x, '/')[-2])
scraped_df['id_2'] = scraped_df_2['movie_url_2'].apply(lambda x: split_str_list(x, '/')[-2])

In [63]:
#movies.drop('title_2', axis=1, inplace=True)
movies = movies.merge(scraped_df[['movie_id', 'id', 'id_2']], on='movie_id', how='left')

In [77]:
movies_imdb = pd.read_csv('ml-100k/title.basics.tsv.gz', sep='\t', na_values= '\\N')
movies_imdb = movies_imdb[['tconst', 'startYear', 'titleType']]

In [81]:
movies = movies.merge(movies_imdb, left_on='id', right_on='tconst', how='left').drop('tconst', axis=1)
movies = movies.merge(movies_imdb, left_on='id_2', right_on='tconst', how='left').drop('tconst', axis=1)

out of the two ids, we keep the one that has a release date closer to the original one.

In [83]:
movies['best_id'] = np.where(np.abs(movies['release_date'].astype(float) - movies['startYear_x']) > \
                np.abs(movies['release_date'].astype(float) - movies['startYear_y']), movies['id_2'], movies['id'])

movies['best_year'] = np.where(np.abs(movies['release_date'].astype(float) - movies['startYear_x']) > \
                np.abs(movies['release_date'].astype(float) - movies['startYear_y']), movies['startYear_y'],\
                               movies['startYear_x'])

movies['title_type'] = np.where(np.abs(movies['release_date'].astype(float) - movies['startYear_x']) > \
                np.abs(movies['release_date'].astype(float) - movies['startYear_y']), movies['titleType_y'],\
                               movies['titleType_x'])

In [85]:
movies.drop(['startYear_x', 'titleType_x', 'startYear_y', 'titleType_y'], axis=1, inplace=True)

We replaced manually the movies that seemed that had a difference of more than one year with the original year of the imdb dataset and those whose titleType was not movie

In [88]:
movies.set_value(22, 'best_id', 'tt0075314')
movies.set_value(70, 'best_id', 'tt0110357')
movies.set_value(113, 'best_id', np.nan)
movies.set_value(115, 'best_id', 'tt0112701')
movies.set_value(127, 'best_id', 'tt0104558')
movies.set_value(168, 'best_id', 'tt0108598')
movies.set_value(180, 'best_id', 'tt0086190')
movies.set_value(188, 'best_id', 'tt0104361')
movies.set_value(194, 'best_id', 'tt0088247')
movies.set_value(212, 'best_id', 'tt0091867')

movies.set_value(252, 'best_id', 'tt0114134')
movies.set_value(279, 'best_id', 'tt0118055')
movies.set_value(350, 'best_id', 'tt0118643')
movies.set_value(358, 'best_id', 'tt0118647')
movies.set_value(375, 'best_id', 'tt0110066')
movies.set_value(398, 'best_id', 'tt0108333')
movies.set_value(407, 'best_id', 'tt0112691')
movies.set_value(421, 'best_id', 'tt0115491')
movies.set_value(423, 'best_id', 'tt0115885')
movies.set_value(428, 'best_id', 'tt0043456')

movies.set_value(432, 'best_id', 'tt0097493')
movies.set_value(436, 'best_id', 'tt0103678')
movies.set_value(438, 'best_id', 'tt0106262')
movies.set_value(440, 'best_id', 'tt0078767')
movies.set_value(441, 'best_id', 'tt0099030')
movies.set_value(442, 'best_id', 'tt0056869')
movies.set_value(443, 'best_id', 'tt0051418')
movies.set_value(444, 'best_id', 'tt0037549')
movies.set_value(464, 'best_id', 'tt0110213')
movies.set_value(509, 'best_id', 'tt0047478')

movies.set_value(514, 'best_id', 'tt0082096')
movies.set_value(538, 'best_id', 'tt0119715')
movies.set_value(547, 'best_id', 'tt0110647')
movies.set_value(562, 'best_id', 'tt0112040')
movies.set_value(583, 'best_id', 'tt0108071')
movies.set_value(591, 'best_id', 'tt0114736')
movies.set_value(594, 'best_id', 'tt0116277')
movies.set_value(598, 'best_id', 'tt0106544')
movies.set_value(632, 'best_id', 'tt0029992')
movies.set_value(642, 'best_id', 'tt0106185')

movies.set_value(643, 'best_id', 'tt0096257')
movies.set_value(656, 'best_id', 'tt0056218')
movies.set_value(676, 'best_id', 'tt0116310')
movies.set_value(693, 'best_id', 'tt11333812')
movies.set_value(729, 'best_id', 'tt0110963')
movies.set_value(731, 'best_id', 'tt0106673')
movies.set_value(766, 'best_id', 'tt0112288')
movies.set_value(776, 'best_id', 'tt0112643')
movies.set_value(798, 'best_id', 'tt0112570')
movies.set_value(809, 'best_id', 'tt0111143')

movies.set_value(825, 'best_id', 'tt0117331')
movies.set_value(837, 'best_id', 'tt0093229')
movies.set_value(838, 'best_id', 'tt0094376')
movies.set_value(849, 'best_id', 'tt0117320')
movies.set_value(857, 'best_id', 'tt0115535')
movies.set_value(859, 'best_id', 'tt0092632')
movies.set_value(860, 'best_id', 'tt0091651')
movies.set_value(862, 'best_id', 'tt0065777')
movies.set_value(867, 'best_id', 'tt0175708')
movies.set_value(877, 'best_id', 'tt0120317')

movies.set_value(896, 'best_id', 'tt0128755')
movies.set_value(942, 'best_id', 'tt0110265')
movies.set_value(957, 'best_id', 'tt0110081')
movies.set_value(960, 'best_id', 'tt0255451')
movies.set_value(962, 'best_id', 'tt0108181');

In [89]:
movies.set_value(968, 'best_id', 'tt0063819' ) 
movies.set_value(972, 'best_id', 'tt0113212' )
movies.set_value(986, 'best_id', 'tt0163964' )
movies.set_value(1002, 'best_id', 'tt0120317' )
movies.set_value(1014, 'best_id', 'tt0120118' )
movies.set_value(1029, 'best_id', 'tt0055662' )

movies.set_value(1033, 'best_id', 'tt0117420' )
movies.set_value(1042, 'best_id', 'tt0114268' )
movies.set_value(1057, 'best_id', 'tt0111667' )
movies.set_value(1101, 'best_id', 'tt0118001' )
movies.set_value(1110, 'best_id', 'tt0109655' )

movies.set_value(1120, 'best_id', 'tt0058450' )
movies.set_value(1124, 'best_id', 'tt0055018' )
movies.set_value(1127, 'best_id', 'tt0113283' )
movies.set_value(1128, 'best_id', 'tt0109424' )
movies.set_value(1171, 'best_id', 'tt0032143' )

movies.set_value(1191, 'best_id', 'tt0106473' ) 
movies.set_value(1198, 'best_id', 'tt0109592' )
movies.set_value(1200, 'best_id', 'tt0116992' )
movies.set_value(1201, 'best_id', 'tt0109255' )
movies.set_value(1204, 'best_id', 'tt0117582' )

movies.set_value(1234, 'best_id', 'tt0109266' )
movies.set_value(1235, 'best_id', 'tt0119845' )
movies.set_value(1247, 'best_id', 'tt6130672' )
movies.set_value(1251, 'best_id', 'tt7472500' )
movies.set_value(1285, 'best_id', 'tt0358135' )

movies.set_value(1295, 'best_id', 'tt0107212' )
movies.set_value(1303, 'best_id', 'tt0117169' )
movies.set_value(1311, 'best_id', 'tt6154608' )
movies.set_value(1320, 'best_id', 'tt0114047' )
movies.set_value(1323, 'best_id', 'tt0110374' )

movies.set_value(1327, 'best_id', 'tt0110712' )
movies.set_value(1328, 'best_id', 'tt0125877' );

In [90]:
movies.set_value(1333, 'best_id','tt0111237') 
movies.set_value(1345, 'best_id','tt0104109')
movies.set_value(1357, 'best_id','tt0645145')
movies.set_value(1358, 'best_id','tt4275988')
movies.set_value(1360, 'best_id','tt0117577')
movies.set_value(1363, 'best_id','tt0115684')
movies.set_value(1366, 'best_id','tt0111959')
movies.set_value(1371, 'best_id','tt0109647')
movies.set_value(1372, 'best_id','tt0067152')
movies.set_value(1380, 'best_id','tt0116920')
movies.set_value(1381, 'best_id','tt0058985')
movies.set_value(1385, 'best_id','tt0105569')
movies.set_value(1389, 'best_id','tt0113425')
movies.set_value(1411, 'best_id','tt0113596')
movies.set_value(1416, 'best_id','tt0105660')
movies.set_value(1419, 'best_id','tt0568382')
movies.set_value(1422, 'best_id','tt1520211')
movies.set_value(1427, 'best_id','tt0120238')
movies.set_value(1447, 'best_id','tt0107471')
movies.set_value(1448, 'best_id','tt0048473')
movies.set_value(1471, 'best_id','tt0108500')
movies.set_value(1472, 'best_id','tt0031580')
movies.set_value(1481, 'best_id','tt0113147')
movies.set_value(1490, 'best_id','tt0114706')
movies.set_value(1493, 'best_id','tt0110570')
movies.set_value(1496, 'best_id','tt0116886')
movies.set_value(1503, 'best_id','tt0109255')
movies.set_value(1507, 'best_id','tt0112438')
movies.set_value(1511, 'best_id','tt0052572')
movies.set_value(1515, 'best_id','tt0893566')
movies.set_value(1523, 'best_id','tt0110246')
movies.set_value(1535, 'best_id','tt0109066')
movies.set_value(1538, 'best_id','tt1595680');

In [91]:
movies.set_value(1541, 'best_id', 'tt0017350')
movies.set_value(1547, 'best_id', 'tt0112727')
movies.set_value(1548, 'best_id', 'tt0112899')
movies.set_value(1551, 'best_id', 'tt0113360')
movies.set_value(1556, 'best_id', 'tt0111787')
movies.set_value(1557, 'best_id', 'tt0048956')
movies.set_value(1568, 'best_id', 'tt0094265')
movies.set_value(1583, 'best_id', 'tt0039004')
movies.set_value(1585, 'best_id', 'tt0104684')
movies.set_value(1589, 'best_id', 'tt0112951')
movies.set_value(1590, 'best_id', 'tt0112913')
movies.set_value(1591, 'best_id', 'tt0189582')
movies.set_value(1592, 'best_id', 'tt0101692')
movies.set_value(1593, 'best_id', 'tt0120661')
movies.set_value(1594, 'best_id', 'tt0111173')
movies.set_value(1595, 'best_id', 'tt0113948')
movies.set_value(1599, 'best_id', 'tt0109949')
movies.set_value(1611, 'best_id', 'tt0116845')
movies.set_value(1612, 'best_id', 'tt0114690')
movies.set_value(1625, 'best_id', 'tt0110251')
movies.set_value(1626, 'best_id', 'tt0114936')
movies.set_value(1629, 'best_id', 'tt0111055')
movies.set_value(1633, 'best_id', 'tt0109751')
movies.set_value(1634, 'best_id', 'tt0092123')
movies.set_value(1642, 'best_id', 'tt0112362')
movies.set_value(1644, 'best_id', 'tt0118804')
movies.set_value(1649, 'best_id', 'tt0118804')
movies.set_value(1654, 'best_id', 'tt0109783')
movies.set_value(1655, 'best_id', 'tt0119548')
movies.set_value(1661, 'best_id', 'tt0114303')
movies.set_value(1662, 'best_id', 'tt0114007')
movies.set_value(1666, 'best_id', 'tt0113968')
movies.set_value(1681, 'best_id', 'tt0102855');

Now we can check if some movies have the same link

We already know that some movies are duplicated in the 100k dataset so we remove them to check the real duplicates

In [39]:
movies_temp = movies[~movies.drop('movie_id', axis=1).duplicated()]

In [40]:
imdb_id_count = movies_temp['best_id'].value_counts()
imdb_id_count[imdb_id_count > 1]

tt0118163    2
tt0116583    2
tt0109255    2
tt0114436    2
Name: best_id, dtype: int64

We handle them manually

In [41]:
movies.set_value(1388, 'best_id', 'tt0117076')
movies.set_value(1240, 'best_id', 'tt0118064')
movies.set_value(1546, 'best_id', 'tt0114435');